# 0.1. imports

In [1]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

# 0.2. config

In [2]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

# 0.3. load `pylcaio`

In [3]:
sys.path.append(str(Path(Path.home(), config['path_dir_repo_pylcaio'])))
import pylcaio 

# 0.4. file paths
## 0.4.1. names

In [4]:
%%capture
print(ecoinvent_version_name := config['ecoinvent_version_name'])

## 0.4.2. directories

In [5]:
%%capture
print(path_dir_data := Path(Path.home(), config['path_dir_data']))
print(path_dir_data_raw := Path(path_dir_data, config['path_dir_data_raw']))
print(path_dir_data_processed := Path(path_dir_data, config['path_dir_data_processed']))
print(path_dir_processing_logs := Path(path_dir_data_processed, config['path_dir_processing_logs']))
# ecoinvent
print(path_dir_ecoinvent := Path(path_dir_data_raw, ecoinvent_version_name))

## 0.4.3. files

In [8]:
%%capture
print(path_file_ecoinvent_characterization := Path(path_dir_data_raw, config['ecoinvent_characterization_file']))

# 1.1. load `Ecoinvent`
## 1.1.1 read ecoinvent and write to pickle
⏳ ~12min

In [15]:
parser = e2m.Ecospold2Matrix(
    sys_dir = str(path_dir_ecoinvent), # passing a Posix Path object breaks the functionality
    characterisation_file = path_file_ecoinvent_characterization,
    project_name = ecoinvent_version_name,
    out_dir = path_dir_data_processed,
    positive_waste = False,
    nan2null = True
)
parser.save_interm = False
parser.prefer_pickles = True

2023-01-03 14:48:34,407 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Ecospold2Matrix Processing
2023-01-03 14:48:34,417 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Current git commit: 9a062cd045c2285999f296ef9a524bf3da58c6a7
2023-01-03 14:48:34,418 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Project name: ecoinvent_3.5_cutoff_ecoSpold02
2023-01-03 14:48:34,419 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Unit process and Master data directory: /Users/michaelweinold/data/data_raw/ecoinvent_3.5_cutoff_ecoSpold02
2023-01-03 14:48:34,419 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Data saved in: /Users/michaelweinold/data/data_processed
2023-01-03 14:48:34,419 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Replace Not-a-Number instances with 0.0 in all matrices
2023-01-03 14:48:34,420 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Pickle intermediate results to files
2023-01-03 14:48:34,420 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Order processes based on: ISIC, activityName
2023-01-03 14:48:34,420 - e

In [16]:
parser.ecospold_to_Leontief(
    fileformats = 'Pandas',
    with_absolute_flows = True
)

2023-01-03 14:48:41,554 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Products extracted from IntermediateExchanges.xml with SHA-1 of b2c87a5bf5982a60515a6e1160e43c620a218369
2023-01-03 14:48:55,012 - ecoinvent_3.5_cutoff_ecoSpold02 - WARNING - Removed 1148 duplicate rows from activity_list, see duplicate_activity_list.csv.
2023-01-03 14:48:55,068 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Activities extracted from ActivityIndex.xml with SHA-1 of 3ac94e9826a9a031ff2e0bfbdceeecaeb72a9117
2023-01-03 14:48:55,093 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Processing 16022 files in /Users/michaelweinold/data/data_raw/ecoinvent_3.5_cutoff_ecoSpold02/datasets
2023-01-03 14:49:51,153 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Processing 16022 files - this may take a while ...
2023-01-03 14:51:07,770 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Elementary flows extracted from ElementaryExchanges.xml with SHA-1 of 0caa74a71870c1432557a91516a6da63d319b594
2023-01-03 14:51:07,791 - ecoinvent_3.5_cutof

starting characterisation


2023-01-03 14:51:34,188 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Will use column 7, named CF_35, for characterisation factors
2023-01-03 14:51:34,189 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Starting characterisation matching
2023-01-03 14:51:38,078 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Characterisation matching done. C matrix created
2023-01-03 14:51:38,120 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Starting to export to file
2023-01-03 14:51:38,121 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - about to write to file
2023-01-03 14:55:08,571 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Final, symmetric, normalized matrices saved in /Users/michaelweinold/data/data_processed/ecoinvent_3.5_cutoff_ecoSpold02Pandas_symmNorm.gz.pickle with SHA-1 of 9f1d76d633cca6be7f1e2f9e2fecf010c70033b0
2023-01-03 14:58:40,225 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Final, symmetric, scaled-up flow matrices saved in /Users/michaelweinold/data/data_processed/ecoinvent_3.5_cutoff_ecoSpold02Pandas_symm

## 1.1.2 remove all files other than the pickled dictionaries of `pd.DataFrame`s

In [31]:
Path.unlink(Path.cwd() / (ecoinvent_version_name + '_characterisation.db'), missing_ok = True)
Path.unlink(Path.cwd() / 'C_long', missing_ok = True)